# Agent Example: Agent with Tools
This Notebook uses an efficient agent approach with tools, but does not utilize LangGraph

### Notebook setup

In [2]:
# %pip install --upgrade langchain langsmith langchain_openai

In [3]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OpenAI keys from .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORGANIZATION")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Demos"

# Initialize OpenWeatherMap
OPENWEATHERMAP_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")

### Set up the Tools


In [4]:
# Import things that are needed generically for tools below
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

In [5]:
import requests


class City(BaseModel):
    city: str = Field(description="City")
    country: str = Field(description="Country code")


def get_current_weather(city: str, country: str) -> int:
    response = requests.get(
        f"http://api.openweathermap.org/data/2.5/weather?q={city},{country}&appid={OPENWEATHERMAP_API_KEY}"
    )
    data = response.json()
    temp_kelvin = data["main"]["temp"]
    temp_fahrenheit = (temp_kelvin - 273.15) * 9 / 5 + 32
    return int(temp_fahrenheit)


weather = StructuredTool.from_function(
    func=get_current_weather,
    name="Get_Weather",
    description="Get the current temperature from a city, in Fahrenheit",
    args_schema=City,
    return_direct=False,
)

In [6]:
class DifferenceInput(BaseModel):
    minuend: int = Field(
        description="The number from which another number is to be subtracted"
    )
    subtrahend: int = Field(description="The number to be subtracted")


def get_difference(minuend: int, subtrahend: int) -> int:
    return minuend - subtrahend


difference = StructuredTool.from_function(
    func=get_difference,
    name="Difference",
    description="Get the difference between two numbers",
    args_schema=DifferenceInput,
    return_direct=False,
)

In [7]:
tools = [weather, difference]

### Set up the agent


In [8]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)

system_template = """
You are a helpful assistant.
"""

human_template = "{input}"

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template(
            input_variables=["input"], template=human_template
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(model, tools, prompt)

### Let's run this agent


In [11]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_config({"run_name": "Agent with Tools"})

In [12]:
agent_executor.invoke(
    {
        "input": "Where is it warmest: Austin, Texas; Tokyo; or Seattle? And by how much is it warmer than the other cities?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `Get_Weather` with `{'city': 'Austin', 'country': 'US'}`


71
Invoking: `Get_Weather` with `{'city': 'Tokyo', 'country': 'JP'}`


39
Invoking: `Get_Weather` with `{'city': 'San Francisco', 'country': 'US'}`


52
Invoking: `Difference` with `{'minuend': 71, 'subtrahend': 39}`


32The warmest location is Austin, Texas with a temperature of 71°F. The temperature difference between Austin and Tokyo is 32°F, and the temperature difference between Austin and San Francisco is 19°F.

> Finished chain.


{'input': 'Where is it warmest: Austin, Texas; Tokyo; or San Francisco? And by how much?',
 'output': 'The warmest location is Austin, Texas with a temperature of 71°F. The temperature difference between Austin and Tokyo is 32°F, and the temperature difference between Austin and San Francisco is 19°F.'}